In [138]:
from pyspark import SparkContext, SparkConf
import pandas as pd
import re
from operator import add
import numpy as np

In [ ]:
import os
data1 = pd.read_csv("_Answers.csv",encoding="latin-1", iterator = True, chunksize=100)
# print(data1)
# data1.apply(lambda row: row.replace("\n", " "))
# data1.to_csv("aaa.csv",encoding='utf-8')
# print(data1.iloc[0])
hasHeader = False
os.remove("aaa.csv")
for chunk in data1:
    #print(chunk)
    chunk["Body"] = chunk["Body"].map(lambda body: body.replace("\n", " "))
    chunk["Body"] = chunk["Body"].map(lambda body: body.replace(",", ";"))
    #chunk["Title"] = chunk["Title"].map(lambda body: body.replace(",", ";"))
    #print(chunk)
    #print(chunk["Body"])
    if hasHeader:
        chunk.to_csv("aaa.csv", mode = "a", encoding="utf-8", header=False, index = False)
    else:
        chunk.to_csv("aaa.csv", mode = "a", encoding="utf-8", header=True, index = False)

    #break
#     print(type(chunk))
#     for index, row in chunk.iterrows():
#         print(row)
#         print(row["Body"])
#         chunk.iloc[index]["Body"] = chunk.iloc[index]["Body"].replace("\n", " ")
#         print(chunk.iloc[index]["Body"])
#         break
# data1.to_csv("aaa.csv",encoding='utf-8')
# data1 = pd.read_csv("aaa.csv")
# data1.replace('\n','')
# data1.to_csv("bbb.csv")
print("finish")
df = spark.read.csv("aaa.csv",header=True);
print(df.take(3))

In [2]:
sc = SparkContext.getOrCreate()

In [29]:
questionFile=sc.textFile("Questions.csv")
answerFile=sc.textFile("Answers.csv")
tagFile = sc.textFile("Tags.csv")
stopWordsFile = sc.textFile("stopwords.txt")
print("finished")

finished


In [30]:
questionWithHeader = questionFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)
header = questionWithHeader.first() #extract header
question = questionWithHeader.filter(lambda x: x != header)
print(question.take(2))

[['90', '58.0', '2008-08-01T14:41:24Z', '2012-12-26T03:45:49Z', '144', 'Good branching and merging tutorials for TortoiseSVN?', '"<p>Are there any really good tutorials explaining <a href=""http://svnbook.red-bean.com/en/1.8/svn.branchmerge.html"" rel=""nofollow"">branching and merging</a> with Apache Subversion? </p>  <p>All the better if it\'s specific to TortoiseSVN client.</p> "'], ['120', '83.0', '2008-08-01T15:50:08Z', '', '21', 'ASP.NET Site Maps', "<p>Has anyone got experience creating <strong>SQL-based ASP.NET</strong> site-map providers?</p>  <p>I've got the default XML file <code>web.sitemap</code> working properly with my Menu and <strong>SiteMapPath</strong> controls; but I'll need a way for the users of my site to create and modify pages dynamically.</p>  <p>I need to tie page viewing permissions into the standard <code>ASP.NET</code> membership system as well.</p> "]]


In [31]:
stopWordsList = stopWordsFile.collect()


In [139]:
questionLower = question.map(lambda x: (x[0], (x[5]+x[6]).lower()))
print(questionLower.take(2))

[('80', 'sqlstatement.execute() - multiple queries in one statement"<p>i\'ve written a database generation script in <a href=""http://en.wikipedia.org/wiki/sql"">sql</a> and want to execute it in my <a href=""http://en.wikipedia.org/wiki/adobe_integrated_runtime"">adobe air</a> application:</p>  <pre><code>create table trole (       roleid integer primary key       ;rolename varchar(40) ); create table tfile (     fileid integer primary key     ;filename varchar(50)     ;filedescription varchar(500)     ;thumbnailid integer     ;fileformatid integer     ;categoryid integer     ;isfavorite boolean     ;dateadded date     ;globalaccesscount integer     ;lastaccesstime date     ;downloadcomplete boolean     ;isnew boolean     ;isspotlight boolean     ;duration varchar(30) ); create table tcategory (     categoryid integer primary key     ;categoryname varchar(50)     ;parent_categoryid integer ); ... </code></pre>  <p>i execute this in adobe air using the following methods:</p>  <pre><cod

In [35]:
def f(x): return re.sub('[^a-zA-Z]+', ' ', x)
questionLower = questionLower.mapValues(f)
print(questionLower.take(2))

[('90', 'good branching and merging tutorials for tortoisesvn p are there any really good tutorials explaining a href http svnbook red bean com en svn branchmerge html rel nofollow branching and merging a with apache subversion p p all the better if it s specific to tortoisesvn client p '), ('120', 'asp net site maps p has anyone got experience creating strong sql based asp net strong site map providers p p i ve got the default xml file code web sitemap code working properly with my menu and strong sitemappath strong controls but i ll need a way for the users of my site to create and modify pages dynamically p p i need to tie page viewing permissions into the standard code asp net code membership system as well p ')]


In [38]:
def f2(x): return x.split(" ")
questionPairRaw = questionLower.flatMapValues(f2)
print(questionPairRaw.take(10))

[('90', 'good'), ('90', 'branching'), ('90', 'and'), ('90', 'merging'), ('90', 'tutorials'), ('90', 'for'), ('90', 'tortoisesvn'), ('90', 'p'), ('90', 'are'), ('90', 'there')]


In [140]:
questionPair=questionPairRaw.filter(lambda x: x[1] not in stopWordsList)
print(questionPair.take(10))

[('80', 'sqlstatement'), ('80', 'multiple'), ('80', 'queries'), ('80', 'statement'), ('80', 've'), ('80', 'written'), ('80', 'database'), ('80', 'generation'), ('80', 'script'), ('80', 'href')]


In [41]:
questionPairfilter=questionPair.filter(lambda x:x[1]!="")
print(questionPairfilter.take(10))

[('90', 'good'), ('90', 'branching'), ('90', 'merging'), ('90', 'tutorials'), ('90', 'tortoisesvn'), ('90', 'good'), ('90', 'tutorials'), ('90', 'explaining'), ('90', 'href'), ('90', 'http')]


In [44]:
questionPairStage1 = questionPairfilter.map(lambda x:(x[0]+"@"+x[1],1)).reduceByKey(add)
print(questionPairStage1.take(10))

[('90@good', 2), ('90@branching', 2), ('90@merging', 2), ('90@http', 1), ('90@svnbook', 1), ('90@red', 1), ('90@bean', 1), ('90@en', 1), ('90@svn', 1), ('90@branchmerge', 1)]


In [45]:
print(questionPairStage1.take(5))

[('90@good', 2), ('90@branching', 2), ('90@merging', 2), ('90@http', 1), ('90@svnbook', 1)]


In [153]:
N = len(questionPair.countByKey())
print(N)

7


In [154]:
print(questionPairStage1.take(5))

[('90@good', 2), ('90@branching', 2), ('90@merging', 2), ('90@http', 1), ('90@svnbook', 1)]


In [155]:
questionPairStage2MapPhase = questionPairStage1.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
print(questionPairStage2MapPhase.take(5))

[('good', '90=2'), ('branching', '90=2'), ('merging', '90=2'), ('http', '90=1'), ('svnbook', '90=1')]


In [156]:
stage2Map = questionPairStage2MapPhase.countByKey()
print("finish")

finish


In [157]:
print(stage2Map.get('statement'))

None


In [158]:
questionPairStage2 = questionPairStage2MapPhase.map(lambda x:(x[0]+"@"+x[1].split("=")[0],(1+np.log(int(x[1].split("=")[1])))*np.log(N/stage2Map.get(x[0]))))
print(questionPairStage2.take(5))

[('good@90', 2.1211120880178402), ('branching@90', 3.2947122824959867), ('merging@90', 3.2947122824959867), ('http@90', 1.252762968495368), ('svnbook@90', 1.9459101490553132)]


In [159]:
questionPairStage3MapPhase = questionPairStage2.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
print(questionPairStage3MapPhase.take(5))

[('90', 'good=2.1211120880178402'), ('90', 'branching=3.2947122824959867'), ('90', 'merging=3.2947122824959867'), ('90', 'http=1.252762968495368'), ('90', 'svnbook=1.9459101490553132')]


In [160]:
def f3_1(a,b): return float(a)+float(b.split("=")[1])*float(b.split("=")[1])
def f3_2(a,b): return float(a)+float(b)
questionPairStage3AggByKey = questionPairStage3MapPhase.aggregateByKey(0.0,f3_1,f3_2)
print(questionPairStage3AggByKey.take(5))

[('180', 47.723071638474686), ('330', 254.08020392404381), ('470', 99.02189599792834), ('90', 103.42265481514247), ('120', 158.1646965191352)]


In [161]:
print(questionPairStage3AggByKey.lookup('90'))

[103.42265481514247]


In [162]:
questionPairStage3AggByKeysqr = questionPairStage3AggByKey.map(lambda x:(x[0],np.sqrt(x[1])))
print(questionPairStage3AggByKeysqr.lookup('90'))

[10.169692955794805]


In [163]:
stage3Map = questionPairStage3AggByKeysqr.collectAsMap()
print(stage3Map.get('90'))

10.169692955794805


In [89]:
questionPairStage3 = questionPairStage3MapPhase.map(lambda x:(x[0],x[1].split("=")[0]+"="+str(float(x[1].split("=")[1])/stage3Map.get(x[0]))))
print(questionPairStage3.take(500))

[('90', 'good=0.2704304029025912'), ('90', 'branching=0.310874638538734'), ('90', 'merging=0.310874638538734'), ('90', 'http=0.15972055235809823'), ('90', 'svnbook=0.18360756944704817'), ('90', 'red=0.18360756944704817'), ('90', 'bean=0.18360756944704817'), ('90', 'en=0.15972055235809823'), ('90', 'svn=0.18360756944704817'), ('90', 'branchmerge=0.18360756944704817'), ('90', 'html=0.18360756944704817'), ('90', 'rel=0.18360756944704817'), ('90', 'nofollow=0.18360756944704817'), ('90', 'apache=0.18360756944704817'), ('90', 'better=0.18360756944704817'), ('90', 'client=0.15972055235809823'), ('120', 'asp=0.3014508178375114'), ('120', 'net=0.23929142028143402'), ('120', 'maps=0.14364292988526622'), ('120', 'experience=0.14364292988526622'), ('120', 'creating=0.12495524107586918'), ('120', 'strong=0.3427743135999506'), ('120', 'providers=0.14364292988526622'), ('120', 've=0.11402364389722552'), ('120', 'file=0.12495524107586918'), ('120', 'sitemap=0.14364292988526622'), ('120', 'properly=0.1

In [164]:
num_keywords = 7
def f4_1(a,b):
    c=[]
    for i in range(num_keywords):
        if float(b.split("=")[1])>float(a[i].split("=")[1]):
            a[i]=b
            break
    return a
def f4_2(a,b):
    for i in range(num_keywords): #b
        for j in range(num_keywords): #a
            if float(b[i].split("=")[1])>float(a[j].split("=")[1]):
                a[j] = b[i]
                break
    return a
questionPairStage4 = questionPairStage3.aggregateByKey(["a=0","b=0","c=0","d=0","e=0","f=0","g=0"],f4_1,f4_2)
print(questionPairStage4.take(5))

[('180', ['color=0.2816816712576253', 'wheels=0.2816816712576253', 'times=0.2816816712576253', 'colors=0.2816816712576253', 'pseudo=0.2816816712576253', 'solved=0.2816816712576253', 'stuck=0.2816816712576253']), ('330', ['classes=0.35963078843880864', 'video=0.25619436907914805', 'workhorse=0.20669600384213413', 'book=0.20669600384213413', 'mine=0.20669600384213413', 'play=0.12207798956602055', 'pause=0.12207798956602055']), ('470', ['homegrown=0.33109443109656533', 'services=0.33109443109656533', 'writing=0.1955497046553674', 'consume=0.1955497046553674', 'examples=0.1955497046553674', 'consuming=0.1955497046553674', 'generated=0.1955497046553674']), ('90', ['branching=0.3239736240629195', 'merging=0.3239736240629195', 'tutorials=0.3239736240629195', 'tortoisesvn=0.3239736240629195', 'bean=0.1913440413111501', 'svn=0.1913440413111501', 'branchmerge=0.1913440413111501']), ('120', ['strong=0.36922578455308835', 'site=0.32471340559703277', 'experience=0.1547276775945656', 'providers=0.15

In [165]:
outMap=questionPairStage4.collectAsMap()
print(outMap)

{'180': ['color=0.2816816712576253', 'wheels=0.2816816712576253', 'times=0.2816816712576253', 'colors=0.2816816712576253', 'pseudo=0.2816816712576253', 'solved=0.2816816712576253', 'stuck=0.2816816712576253'], '330': ['classes=0.35963078843880864', 'video=0.25619436907914805', 'workhorse=0.20669600384213413', 'book=0.20669600384213413', 'mine=0.20669600384213413', 'play=0.12207798956602055', 'pause=0.12207798956602055'], '470': ['homegrown=0.33109443109656533', 'services=0.33109443109656533', 'writing=0.1955497046553674', 'consume=0.1955497046553674', 'examples=0.1955497046553674', 'consuming=0.1955497046553674', 'generated=0.1955497046553674'], '90': ['branching=0.3239736240629195', 'merging=0.3239736240629195', 'tutorials=0.3239736240629195', 'tortoisesvn=0.3239736240629195', 'bean=0.1913440413111501', 'svn=0.1913440413111501', 'branchmerge=0.1913440413111501'], '120': ['strong=0.36922578455308835', 'site=0.32471340559703277', 'experience=0.1547276775945656', 'providers=0.15472767759

In [166]:
tagWithHeader = tagFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)
header = tagWithHeader.first() #extract header
tag = tagWithHeader.filter(lambda x: x != header)
print(tag.take(2))

[['80', 'flex'], ['80', 'actionscript-3']]


In [167]:
tagLower = tag.map(lambda x: (x[0], x[1].lower()))
print(tagLower.take(2))

[('80', 'flex'), ('80', 'actionscript-3')]


In [168]:
def f0(x): return re.sub('[^a-zA-Z]+', '', x)
tagLower = tagLower.mapValues(f0)
print(tagLower.take(2))

[('80', 'flex'), ('80', 'actionscript')]


In [169]:
tag_1000 = tagLower.map(lambda x: (x[0], x[1]+"=1000"))
print(tag_1000.take(2))

[('80', 'flex=1000'), ('80', 'actionscript=1000')]


In [170]:
questionPairStage5=questionPairStage4.map(lambda x:(x[0],x[1][0]+":"+x[1][1]+":"+x[1][2]+":"+x[1][3]+":"+x[1][4]+":"+x[1][5]+":"+x[1][6]) )
print(questionPairStage5.take(5))

[('180', 'color=0.2816816712576253:wheels=0.2816816712576253:times=0.2816816712576253:colors=0.2816816712576253:pseudo=0.2816816712576253:solved=0.2816816712576253:stuck=0.2816816712576253'), ('330', 'classes=0.35963078843880864:video=0.25619436907914805:workhorse=0.20669600384213413:book=0.20669600384213413:mine=0.20669600384213413:play=0.12207798956602055:pause=0.12207798956602055'), ('470', 'homegrown=0.33109443109656533:services=0.33109443109656533:writing=0.1955497046553674:consume=0.1955497046553674:examples=0.1955497046553674:consuming=0.1955497046553674:generated=0.1955497046553674'), ('90', 'branching=0.3239736240629195:merging=0.3239736240629195:tutorials=0.3239736240629195:tortoisesvn=0.3239736240629195:bean=0.1913440413111501:svn=0.1913440413111501:branchmerge=0.1913440413111501'), ('120', 'strong=0.36922578455308835:site=0.32471340559703277:experience=0.1547276775945656:providers=0.1547276775945656:sitemap=0.1547276775945656:properly=0.1547276775945656:menu=0.1547276775945

In [171]:
questionPairStage6=questionPairStage5.union(tag_1000)
def f6_1(a,b):
    return a+":"+b
def f6_2(a,b):
    return a+b
questionPairStage6=questionPairStage6.aggregateByKey("",f6_1,f6_2).map(lambda x: (x[0], x[1][1:]))
print(questionPairStage6.take(1))

[('180', 'color=0.2816816712576253:wheels=0.2816816712576253:times=0.2816816712576253:colors=0.2816816712576253:pseudo=0.2816816712576253:solved=0.2816816712576253:stuck=0.2816816712576253:algorithm=1000:languageagnostic=1000:colors=1000:colorspace=1000')]


In [172]:
def f7(x):
    listx= x[1].split(":")
    listx.sort(key=lambda item:float(item.split("=")[1]),reverse=True)
    output=""
    for i in range(num_keywords):
        output+=":"+listx[i]
    return (x[0],output)
questionPairStage7=questionPairStage6.map(f7).map(lambda x: (x[0], x[1][1:]))
print(questionPairStage7.take(2))

[('180', 'algorithm=1000:languageagnostic=1000:colors=1000:colorspace=1000:color=0.2816816712576253:wheels=0.2816816712576253:times=0.2816816712576253'), ('330', 'c=1000:oop=1000:class=1000:nestedclass=1000:classes=0.35963078843880864:video=0.25619436907914805:workhorse=0.20669600384213413')]
